In [17]:
# Blibiotecas

# Modelation
import numpy as np
import pandas as pd
from functions import get_files_name, upload_sheets

A base de dados selecionada para se projeto, que povem do SNIS(Sistema nacional de informações sobre Saneamento) contem 2 divisões de dados:
 - RS - Residuos solidos
 - AP - Aguas Pluviais

Vamos iniciar o tratamento pela base AE

# Extraindo e manipulando os dados de Agua e Esgoto

Nessa base de dados existem varias planilhas com indicadores de Agua e Esgoto. As planilhas possuem a mesma nomenclatura e colunas, porém foram separadas porque cada uma representa uma compania de agua.


In [18]:
df_base = pd.read_excel('base_de_dados/2Planilhas_AE2021/Planilhas_AE2021/Planilha_AE2021_Completa_Regionais/Planilha_AE_Indicadores_CAESB-53001000.xls', skiprows=7)
df_base.head()


,Código do município,Município,UF,Código da região,Código do prestador de serviços,Nome do prestador de serviços,Sigla,Abrangência,Natureza jurídica do prestador de serviços,Unnamed: 9,...,Incidência das análises de coliformes totais fora do padrão,Liquidez corrente,Liquidez geral,Grau de endividamento,Margem operacional com depreciação,Margem operacional sem depreciação,Margem líquida com depreciação,Margem líquida sem depreciação,Retorno sobre o patrimônio líquido,Composição de exigibilidades
0,-,-,-,-,-,-,-,-,-,-,...,percentual,-,-,-,percentual,percentual,percentual,percentual,percentual,percentual
1,-,-,-,-,-,-,-,-,-,-,...,IN084,IN061,IN062,IN063,IN064,IN068,IN065,IN069,IN066,IN067
2,530010,Brasília,DF,5,53001000,Companhia de Saneamento Ambiental do Distrito ...,CAESB,Regional,Sociedade de economia mista com administração ...,Água e Esgotos,...,0.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,INDICADOR DA BASE AGREGADA:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.9,1.3,1.21,0.6,6.96,10.5,6.8,10.99,8.82,29.59
4,ANO 2020:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.5,1.36,1.22,0.6,17.88,21.39,10.47,14.55,14.45,26.76


In [19]:
# Filtrando somente a linha desejada
df_base = df_base.loc[(df_base['Município'] != '') & (df_base['Município'].notna()) & (df_base['Município'] != '-')]
# dropando colunas indesejadas
columns = ['Liquidez corrente','Liquidez geral','Grau de endividamento','Margem operacional com depreciação','Margem operacional sem depreciação',
          'Margem líquida com depreciação','Margem líquida sem depreciação', 'Retorno sobre o patrimônio líquido', 'Composição de exigibilidades',
          'Nome do prestador de serviços','Sigla', 'Abrangência','Natureza jurídica do prestador de serviços','Código do prestador de serviços',
          'Código da região']
df_filtered = df_base.drop(columns=columns)
df_filtered = df_filtered.fillna(0)
df_filtered = df_filtered.replace('', 0)
df_filtered.head()

,Código do município,Município,UF,Unnamed: 9,Despesa total com os serviços por m3 faturado,Despesa de exploração por m3 faturado,Despesa de exploração por economia,Tarifa média praticada,Tarifa média de água,Tarifa média de esgoto,...,Duração média das intermitências,Duração média dos reparos de extravasamentos de esgotos,Extravasamentos de esgotos por extensão de rede,Duração média dos serviços executados,Índice de conformidade da quantidade de amostra - Cloro Residual,Incidência das análises de cloro residual fora do padrão,Índice de conformidade da quantidade de amostra - Turbidez,Incidência das análises de turbidez fora do padrão,Índice de conformidade da quantidade de amostra - Coliformes Totais,Incidência das análises de coliformes totais fora do padrão
2,530010,Brasília,DF,Água e Esgotos,6.5,5.08,706.83,6.21,6.33,6.07,...,0,32.3,1.68,128.94,209.71,0.39,205,5.15,122.72,0.9



Separando df por tipo de informação contida, porque no dataframe original existem 5 seguimentações de dados:
 - Indicadores do balanço contabio
 - indicadores econôminos-financeiros e administrativos
 - indicadores operacionais - Agua
 - Indicadores operacionais - Esgoto
 - Indicadores sobre Qualidade

Desses todos, nos interessam os 3 ultimos, Agua, Esgoto e Qualidade


In [20]:
# Separando os dataframes por tipo de informação
df_ind_agua = df_filtered.iloc[:,36:58]
df_ind_esgoto = df_filtered.iloc[:,58:66]
df_ind_qualidade = df_filtered.iloc[:,66:78]
df_base = df_base.iloc[:, :3]

# Criando um dataframe com as informacoes da respectiva cidade indicada
df_ind_agua = pd.merge(df_base, df_ind_agua, left_index=True, right_index=True, how='outer')
df_ind_esgoto = pd.merge(df_base, df_ind_esgoto, left_index=True, right_index=True, how='outer')
df_ind_qualidade = pd.merge(df_base, df_ind_qualidade, left_index=True, right_index=True, how='outer')

Para ter base de comparação, vamos fazer o mesmo procedimento para todas as outras empresas que tem dados disponíveis nessa base, assim poderemos verificar como o Distrito Federal esta rankeado nos indices disponíveis

### Criando loop para ler todos os arquivos e colher as informações que necessitamos

In [21]:
folder = "base_de_dados/2Planilhas_AE2021/Planilhas_AE2021/Planilha_AE2021_Completa_Regionais"
columns = ['Liquidez corrente','Liquidez geral','Grau de endividamento','Margem operacional com depreciação','Margem operacional sem depreciação',
        'Margem líquida com depreciação','Margem líquida sem depreciação', 'Retorno sobre o patrimônio líquido', 'Composição de exigibilidades',
        'Nome do prestador de serviços','Sigla', 'Abrangência','Natureza jurídica do prestador de serviços','Código do prestador de serviços',
        'Código da região']
df_final_ind_agua = pd.DataFrame()
df_final_ind_esgoto = pd.DataFrame()
df_final_ind_qualidade = pd.DataFrame()

files_name = get_files_name(folder)
# Loop for para mapeamento das planilhas e concatenação dos dados
for file in files_name:
    df_states = pd.read_excel(f"{folder}/{file}", skiprows=7)
    df_states = df_states.loc[(df_states['Município'] != '') & (df_states['Município'].notna()) & (df_states['Município'] != '-')]
    # dropando colunas indesejadas
    df_filtered = df_states.drop(columns=columns)
    # Trocando valores None por 0
    df_filtered = df_filtered.fillna(0)
    df_filtered = df_filtered.replace('',0)
    # Separando os dataframes por tipo de informação
    df_ind_agua = df_filtered.iloc[:,36:58]
    df_ind_esgoto = df_filtered.iloc[:,58:66]
    df_ind_qualidade = df_filtered.iloc[:,66:78]
    df_filtered = df_filtered.iloc[:, :3]
    # Criando um dataframe com as informacoes da respectiva cidade indicada
    df_ind_agua = pd.merge(df_filtered, df_ind_agua, left_index=True, right_index=True, how='outer')
    df_ind_esgoto = pd.merge(df_filtered, df_ind_esgoto, left_index=True, right_index=True, how='outer')
    df_ind_qualidade = pd.merge(df_filtered, df_ind_qualidade, left_index=True, right_index=True, how='outer')
    # realizando merge dos dataframes
    df_final_ind_agua = pd.concat([df_final_ind_agua,df_ind_agua])
    df_final_ind_esgoto = pd.concat([df_final_ind_esgoto,df_ind_esgoto])
    df_final_ind_qualidade = pd.concat([df_final_ind_qualidade,df_ind_qualidade])

# Excluindo os dados nan dos dataframes
df_final_ind_agua = df_final_ind_agua.dropna()
df_final_ind_esgoto = df_final_ind_esgoto.dropna()
df_final_ind_qualidade = df_final_ind_qualidade.dropna()
    

### Enviando os dados para a plataforma Google Sheets

In [22]:
# informações para a função de upload
sheet_id = '1psQ_UB3RW70DPeTZ1No-Q9QQxcj6Aa4jB1xPIYHPwVo'
credentials_folder = 'Credentials/credentials_google.json'
sheet_page_agua = 'AE_IND_AGUA'
sheet_page_esg = 'AE_IND_ESG'
sheet_page_qua = 'AE_IND_QUA'
sheet_range = 'A1:AJ1000000'

# Enviando os dados Indicadores de agua    
result = upload_sheets(df_final_ind_agua, sheet_id, sheet_page_agua, sheet_range, credentials_folder)
# # Enviando os dados Indicadores de esgoto    
result = upload_sheets(df_final_ind_esgoto, sheet_id, sheet_page_esg, sheet_range, credentials_folder)
# Enviando os dados Indicadores de qualidade    
result = upload_sheets(df_final_ind_qualidade, sheet_id, sheet_page_qua, sheet_range, credentials_folder)


# Extraindo e manipulando os dados de Aguas Pluviais

Nessa base de dados temos duas planilhas ( Indicadores e Informaçoes), e vamos trata-las separamente porque possuem nomenclaturas e dados totalmente distintos

### Colhendo dados da planilha original - INDICADORES

In [23]:
df_base = pd.read_excel('base_de_dados/Planilhas_AP2021/Tabela de Indicadores_AP2021.xls', sheet_name='Indicadores por município',skiprows=7)
df_ind_ap = df_base.drop([0,1,2]) # Dropando linhas indesejadas
df_ind_ap = df_ind_ap.rename(columns={'Nome':'Município'})
df_ind_ap = df_ind_ap.fillna(0)
df_ind_ap = df_ind_ap.replace('', 0)
df_ind_ap.head()


,Código IBGE,Município,UF,Região,Capital,Crítico,Faixa populacional,Parcela de área urbana em relação à área total,Densidade Demográfica na Área Urbana,Densidade de Domicílios na Área Urbana,...,Parcela de Cursos d’Água Naturais Perenes em Área Urbana com Parques Lineares,Parcela de Cursos d’Água Naturais Perenes com Canalização Aberta,Parcela de Cursos d’Água Naturais Perenes com Canalização Fechada,Parcela de Cursos d’Água Naturais Perenes com Diques,Volume de reservação de águas pluviais por unidade de área urbana,Densidade de captações de águas pluviais na área urbana,Parcela de Domicílios em Situação de Risco de Inundação,Parcela da População Impactada por Eventos Hidrológicos,Índice de Óbitos,Habitantes Realocados em Decorrência de Eventos Hidrológicos
3,1100015.0,Alta Floresta D Oeste,RO,Norte,Não,Sim,1 - De 1 até 30.000 hab.,0.12,15,5,...,0.0,18.5,0.0,0.0,0.0,36,0.6,0.0,0,0
4,1100023.0,Ariquemes,RO,Norte,Não,Sim,3 - De 100.001 até 250.000 hab.,0.97,22,7,...,4.9,4.9,1.9,0.0,0.0,646,0.6,0.1,0,94
5,1100031.0,Cabixi,RO,Norte,Não,Não,1 - De 1 até 30.000 hab.,0.11,14,7,...,0.0,0.0,0.0,0.0,0.0,4,0.0,0.0,0,0
6,1100049.0,Cacoal,RO,Norte,Não,Sim,2 - De 30.001 até 100.000 hab.,0.75,24,11,...,0.0,6.3,15.6,0.0,0.0,27,1.2,0.0,0,12
7,1100056.0,Cerejeiras,RO,Norte,Não,Não,1 - De 1 até 30.000 hab.,5.21,1,0,...,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0,0


Nesse conjunto de dados em especifico, relacionado aos indicadores de água, as bases já estão agrupadas, não necessiando fazer modelagens

### Enviando os dados para a plataforma Google Sheets

In [24]:
# informações para a função de upload
sheet_id = '1psQ_UB3RW70DPeTZ1No-Q9QQxcj6Aa4jB1xPIYHPwVo'
credentials_folder = 'Credentials/credentials_google.json'
sheet_page_ind = 'AP_IND'
sheet_range = 'A1:AJ1000000'

# Enviando os dados de Indicadores
result = upload_sheets(df_ind_ap, sheet_id, sheet_page_ind, sheet_range, credentials_folder)

### Colhendo dados da planilha original - INFORMAÇÕES

In [25]:
df_base = pd.read_excel('base_de_dados/Planilhas_AP2021/Tabela de Informaçoes_AP2021.xlsx', sheet_name='Informações',skiprows=7)
df_base = df_base.drop([0,1,2,3]) # Dropando linhas indesejadas
columns = {'Unnamed: 0': 'Codigo IBGE','Unnamed: 1': 'Município', 'Unnamed: 2':'UF', 'Unnamed: 3':'Região', 'Unnamed: 4':'Capital'}
df_base = df_base.rename(columns=columns)

Nesse conjunto dados as bases também estao agrupadas, porém necessitando de uma fragmentação de colunas, similar ao que fizemos nos dados de AE.
Nesse caso, vamos solicitar somente os dados de informações gerais, para gerar comparações com os indicativos

In [26]:
# Pegando somente as colunas desejadas
df_inf_ap = df_base.iloc[:,:18]
df_inf_ap.replace([np.inf, -np.inf], np.nan, inplace=True)
df_inf_ap.fillna(0, inplace=True)
df_inf_ap['Codigo IBGE'].fillna('', inplace=True)

C:\Users\junio\AppData\Local\Temp\ipykernel_13512\1228036562.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inf_ap.replace([np.inf, -np.inf], np.nan, inplace=True)
C:\Users\junio\AppData\Local\Temp\ipykernel_13512\1228036562.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inf_ap.fillna(0, inplace=True)
C:\Users\junio\AppData\Local\Temp\ipykernel_13512\1228036562.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inf_

In [27]:
# informações para a função de upload
sheet_id = '1psQ_UB3RW70DPeTZ1No-Q9QQxcj6Aa4jB1xPIYHPwVo'
credentials_folder = 'Credentials/credentials_google.json'
sheet_page_inf = 'AP_INF'
sheet_range = 'A1:AJ1000000'

# Enviando os dados de Informações 
result = upload_sheets(df_inf_ap, sheet_id, sheet_page_inf, sheet_range, credentials_folder)